In [135]:
import sqlite3
from datetime import timedelta, datetime
import pandas as pd
import re
import konlpy.tag
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
import csv


# Configuration
database_name = "news_data.db"
table_name = "main_news"

DATA_DIR = "data/news_data/"

# Okt 형태소 분석기 사용
Okt = konlpy.tag.Okt()

# DB로부터 뉴스 크롤링데이터 가져오기
def get_data():
    try:
        con = sqlite3.connect(DATA_DIR + database_name)
        cursor = con.cursor()
    except:
        print("Connection failed!")

    try:
        SQL = "SELECT * from " + table_name
        df = pd.read_sql(SQL, con)
    except Exception as e:
            print(e)
    finally:
        con.commit()
        con.close()
    return df

######### 아래는 혹시 필요할까봐 각각의 과정을 함수화 한 내용입니다 ##########

# 형태소 토큰화
def word_tokenize(df):
    # 형태소 분석 및 단어 추출
    Okt = konlpy.tag.Okt()

    stopwords = ['과', '도', '를', '으로', '자', '에', '와', '한', '하다', '의', '가', '이', '은', '들', '는', '좀', '잘', '수', '것', '만', '있고', '있어서', '있다는'] # 불용어 정의

    df['contents'] = df['contents'].str.replace('[^가-힣A-Za-z0-9]+', ' ', regex=True) # 한글, 영문, 숫자를 제외한 특수문자, 따옴표 등은 정규표현식을 사용하여 모두 제거

    print(df['contents'])

    tokenized_data = []
    for sentence in tqdm(df['contents'][:100]): # 일단 100개만
        # word_tokens = Okt.pos(sentence, stem=False, norm=False)  # Okt 형태소 분석기를 이용한 토큰화
        word_tokens =Okt.morphs(sentence, stem=True)
        word_tokens = [word for word in word_tokens if not word in stopwords] # 불용어 제거
        tokenized_data.append(word_tokens)

    print(tokenized_data)

    return tokenized_data

# 정수 인코딩
def integer_encoding(data):
    print(type(data))
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)

    # 단어 중요도 측정
    threshold = 2
    total_cnt = len(tokenizer.word_index)
    rare_cnt = 0
    total_freq = 0
    rare_freq = 0

    for key, value in tokenizer.word_counts.items():
        total_freq = total_freq + value

        if(value<threshold):
            rare_cnt = rare_cnt +1
            rare_freq = rare_freq +value

    print("단어 집합의 크기:", total_cnt)
    print("등장 빈도가 %s번 이하인 희귀 단어의 수: %s" %(threshold-1, rare_cnt))
    print("단어 집합에서 희귀 단어의 비율:", (rare_cnt/total_cnt)*100)
    print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq/total_freq)*100)

    vocab_size = total_cnt - rare_cnt + 1

    tokenizer = Tokenizer(vocab_size)
    tokenizer.fit_on_texts(data)
    encoded_data = tokenizer.texts_to_sequences(data)

    return encoded_data

# 전체 기사의 길이 분포 확인
def check_data_len(data):
    print("뉴스기사의 최대 길이:",max(len(news) for news in data))
    print("뉴스기사의 평균 길이:", sum(map(len, data))/len(data))
    plt.hist([len(news) for news in data], bins=30)
    plt.xlabel("length of samples")
    plt.ylabel("number of samples")
    plt.show()

# 길이가 기준치 이하인 샘플의 비율 확인
def check_percentage(data, threshold):
    count = 0
    for sentence in data:
        if(len(sentence) <= threshold):
            count +=1
    print("전체 샘플 중 길이가 %s 이하인 데이터의 비율:%s" %(threshold,(count/len(data))*100))

def LSTM(padded_data):
    embedding_dim = 100
    hidden_units = 128
    model = Sequential()
    model.add(Embedding(len(padded_data), embedding_dim))
    model.add(LSTM(hidden_units))
    model.add(Dense(1, activation='sigmoid'))

    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose=1, patience =4)
    mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', verbose=1, save_best_only=True)

In [136]:
# 데이터 불러오기
news_data = get_data()
kospi_data = pd.read_csv("data/kospi_data/data_0601_20220904.csv")
news_data.head()

,date,end,delta,delta_ratio,isup,avrage,high,low,volume,transation_value,total_value
0,2022-09-02,2409.41,-6.20,-0.26,1,2427.70,2432.37,2402.67,391599,7214620.0,1.900000e+09
1,2022-09-01,2415.61,-56.44,-2.28,1,2443.00,2443.61,2415.61,580021,8452318.0,1.900000e+09
2,2022-08-31,2472.05,21.12,0.86,0,2433.47,2473.75,2426.14,436247,9562592.0,1.950000e+09
3,2022-08-30,2450.93,24.04,0.99,0,2441.21,2453.91,2433.48,336555,6826615.0,1.930000e+09
4,2022-08-29,2426.89,-54.14,-2.18,1,2432.06,2432.89,2417.01,448746,7891467.0,1.910000e+09


In [137]:
news_data['date'] = news_data['date'].apply(lambda x: str(x).split(' ')[0])  # 기존 date값에서 년-월-일만 저장
news_data.head()

,id,date,title,contents
0,409,2005-12-19,달러화 약세 월말효과로 유동성 호조,"미래에셋증권, 주식형 펀드 자금유입 매수 여력 보강[프라임경제] 투신권에 월말 효과..."
1,475,2012-12-31,"홈쇼핑·백화점 ""고맙다, 소비""","[머니위크 전보규 기자][[머니위크]유통주, 내년에는 먹구름 걷힐까]유통주는 올해 ..."
2,562,2011-02-10,"이러다 2000선 밑으로?…패닉은 금물, 덜빠진 우등생주'정조준'",■트레이더에게 듣는다▷손은주 대우증권 광교지점 차장외국인 매도가 8000억에 육박했...
3,608,2005-12-23,크리스마스엔 블루칩을 사세요,콜금리 잇단 인상에도 뭉칫돈 지속유입 대형 우량주 강세 예상[프라임경제] 황우석 쇼...
4,612,2011-02-16,당분간 위도 아래도 꽉막힌 장세로…박스권서 살아남으려면,■박옥희 IBK투자증권 스트래티지스트국내 증시의 모멘텀이 없는 가운데 어제에 이어 ...


In [138]:
merge_data = pd.merge(news_data, kospi_data)
#merge_data = merge_data[['contents','isup']] # 기사내용과 레이블 컬럼만 추출
merge_data.head()

,id,date,title,contents,end,delta,delta_ratio,isup,avrage,high,low,volume,transation_value,total_value
0,409,2005-12-19,달러화 약세 월말효과로 유동성 호조,"미래에셋증권, 주식형 펀드 자금유입 매수 여력 보강[프라임경제] 투신권에 월말 효과...",1339.4,18.36,1.39,0,1325.61,1339.48,1320.88,513998,4209738.0,633000000.0
1,24779,2005-12-19,‘황우석 쇼크’ 대안은 3박자 가치주,코스닥시장에서 바이오 테마주 등에 대한 조정세가 이어짐에 따라 ‘실적+배당+수급’의...,1339.4,18.36,1.39,0,1325.61,1339.48,1320.88,513998,4209738.0,633000000.0
2,24836,2005-12-19,[11시40분 시황]주가 ‘黃쇼크’ 탈출 상승세,황우석 쇼크로 급락세를 보였던 서울증시에 반발 매수세가 유입되며 안정을 되찾고 있다...,1339.4,18.36,1.39,0,1325.61,1339.48,1320.88,513998,4209738.0,633000000.0
3,194946,2005-12-19,[윤세욱의 마켓포인트]환율 불안…기간조정 가능성,시장은 이번주에도 조정을 이어갈 전망이다. 원/달러 환율이 1010원대로 하락함에 ...,1339.4,18.36,1.39,0,1325.61,1339.48,1320.88,513998,4209738.0,633000000.0
4,194952,2005-12-19,개미들 투자규모 늘린다,직접투자계좌 최근 3개월여간 32만4000개 증가예탁금도 13조 돌파…“내년에도 증...,1339.4,18.36,1.39,0,1325.61,1339.48,1320.88,513998,4209738.0,633000000.0


In [139]:
X = merge_data['contents'] # 기사내용과 컬럼 추출
y = merge_data['isup'] # 레이블 컬럼 추출

# 사이킷런을 이용하여 학습용 테스트와 테스트용 데이터를 분리
train_data, test_data, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# 데이터 갯수 5000개로 제한
train_data = train_data[:5000]
test_data = test_data[:5000]
y_train = y_train[:5000]
y_test = y_test[:5000]
train_data.head()

65321    25일 아시아 주요 증시가 혼조 마감했다.일본 도쿄증시 닛케이225지수는 이날 전 ...
71302    ⓒ게티이미지뱅크공매도 공시제도가 지난 30일 본격적으로 시행되면서 기관투자자들이 박...
38877    [ 한민수 기자  ] 사진=게티이미지뱅크투자자들의 관심이 미국의 3월 연방공개시장위...
27241    원/달러 환율 상승(원화 약세)으로 원/엔 환율에 민감한 가전, 디스플레이, 핸드셋...
77966    [머니투데이 유윤정 기자]코스피가 닷새만에 반등하며 1640선을 회복했다. 코스피지...
Name: contents, dtype: object

In [140]:
# 형태소 토큰화
stopwords = ['과', '도', '를', '으로', '자', '에', '와', '한', '하다', '의', '가', '이', '은', '들', '는', '좀', '잘', '수', '것', '만', '있고', '있어서', '있다는'] # 불용어 정의

train_data = train_data.str.replace('[^가-힣A-Za-z0-9]+', ' ', regex=True) # 한글, 영문, 숫자를 제외한 특수문자, 따옴표 등은 정규표현식을 사용하여 모두 제거
test_data = test_data.str.replace('[^가-힣A-Za-z0-9]+', ' ', regex=True)

X_train = []
for sentence in tqdm(train_data):
    # word_tokens = Okt.pos(sentence, stem=False, norm=False)  # Okt 형태소 분석기를 이용한 토큰화
    word_tokens =Okt.morphs(sentence, stem=True)
    word_tokens = [word for word in word_tokens if not word in stopwords] # 불용어 제거
    X_train.append(word_tokens)
    
X_test = []
for sentence in tqdm(test_data):
    # word_tokens = Okt.pos(sentence, stem=False, norm=False)  # Okt 형태소 분석기를 이용한 토큰화
    word_tokens =Okt.morphs(sentence, stem=True)
    word_tokens = [word for word in word_tokens if not word in stopwords] # 불용어 제거
    X_test.append(word_tokens)    

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [17:14<00:00,  4.83it/s]


In [141]:
# 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# 단어 중요도 측정
threshold = 3  # 등장빈도 임계값 (3회 미만)
total_cnt = len(tokenizer.word_index) # 단어 수
rare_cnt = 0  # 희귀 단어 수
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value<threshold):
        rare_cnt = rare_cnt +1
        rare_freq = rare_freq +value

print("단어 집합의 크기:", total_cnt)
print("등장 빈도가 %s번 이하인 희귀 단어의 수: %s" %(threshold-1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt/total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq/total_freq)*100)

vocab_size = total_cnt - rare_cnt + 1  # 전체 단어 개수 중 빈도수 3회 미만인 단어는 제거


tokenizer = Tokenizer(vocab_size) # 단어 집합의 크기를 케라스 토크나이저의 인자로 넘겨줌
tokenizer.fit_on_texts(X_train)   # 텍스트 시퀀스를 정수 시퀀스로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

단어 집합의 크기: 49703
등장 빈도가 2번 이하인 희귀 단어의 수: 26275
단어 집합에서 희귀 단어의 비율: 52.86401223266202
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.3933562118204208


In [142]:
# check_data_len(encoded_data)
# check_percentage(encoded_data, 1000)

# 데이터 패딩
max_len = 1000
X_train = pad_sequences(X_train , maxlen, padding='post') # 뒤에 0을 채워서 패딩
X_test = pad_sequences(X_test , maxlen, padding='post')

print(len(X_train))
print(len(y_train))

5000
5000


In [143]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid')) # 활성화 함수로 시그모이드 사용

# 정해진 에포크가 도달하지 못하였더라고 학습을 조기 종료(Early Stopping)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

# 검증 데이터의 정확도가 이전보다 좋아질 경우에만 모델을 저장
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) # 손실 함수로 크로스 엔트로피 함수를 사용
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
63/63 [==============================] - ETA: 0s - loss: 0.6916 - acc: 0.5332 
Epoch 1: val_acc improved from -inf to 0.52800, saving model to best_model.h5
63/63 [==============================] - 1571s 24s/step - loss: 0.6916 - acc: 0.5332 - val_loss: 0.6919 - val_acc: 0.5280
Epoch 2/15
63/63 [==============================] - ETA: 0s - loss: 0.6908 - acc: 0.5383  
Epoch 2: val_acc did not improve from 0.52800
63/63 [==============================] - 11135s 179s/step - loss: 0.6908 - acc: 0.5383 - val_loss: 0.6919 - val_acc: 0.5260
Epoch 3/15
63/63 [==============================] - ETA: 0s - loss: 0.6907 - acc: 0.5420
Epoch 3: val_acc did not improve from 0.52800
63/63 [==============================] - 384s 6s/step - loss: 0.6907 - acc: 0.5420 - val_loss: 0.6925 - val_acc: 0.5270
Epoch 4/15
63/63 [==============================] - ETA: 0s - loss: 0.6931 - acc: 0.5322
Epoch 4: val_acc did not improve from 0.52800
63/63 [==============================] - 428s 7s/step - los

In [144]:
loaded_model = load_model('best_model.h5') # 가장 정확도가 높은 모델이 저장된'best_model.h5'를 로드
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

157/157 [==============================] - 46s 293ms/step - loss: 0.6901 - acc: 0.5414

 테스트 정확도: 0.5414


In [145]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = Okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

today_news = "(서울=뉴스1) 손엄지 기자 = 미국 연방준비제도(Fed, 연준)의 금리인상 속도 조절 기대감과 정부 유동성 공급 조치에 국내 증시가 상승마감했다. 24일 코스피는 전날 대비 23.04p(1.04%) 상승한 2236.16으로 장을 마쳤다. 외국인은 1498억원, 기관은 3246억원 각각 순매수했다. 개인은 4835억원 순매도했다. 선물시장에서 외국인은 662억원 순매도세를 기록했다. 김석환 미래에셋증권 연구원은 △미 연준의 금리인상 속도 조절 기대감 △일본의 외환시장 개입 △정부의 50조원 유동성 공급 조치 영향으로 증시가 상승했다고 분석했다.코스피 시가총액 상위 10개 종목 중 삼성SDI(3.67%), 삼성바이오로직스(3.07%), 삼성전자(2.86%), LG에너지솔루션(2.0%), SK하이닉스(1.44%), LG화학(1.23%), 삼성전자우(0.96%) 등은 상승했다. 기아(-3.83%), 현대차(-3.29%), NAVER(-1.2%) 등은 하락했다.상승 업종은 비금속광물(3.25%), 의료정밀(3.17%), 의약품(2.79%), 건설업(2.45%), 전기전자(2.43%) 등이다. 하락 업종은 운수장비(-1.83%), 보험(-0.97%), 금융업(-0.31%), 서비스업(-0.21%), 화학(-0.13%) 등이다.부동산 프로젝트 파이낸싱(PF) 관련 우려에도 금융당국의 유동성 공급 계획 발표에 건설업이 2% 넘게 올랐다.지난주 미국 뉴욕증시는 연준의 긴축 완화 기대감에 크게 상승했다. 21일(현지시간) 다우 지수는 전장 대비 748.97포인트(2.47%) 급등해 3만1082.56을 기록했고, 스탠다드앤푸어스(S&P)500은 86.97포인트(2.37%) 뛴 3752.75로 체결됐다. 나스닥 지수는 244.87포인트(2.31%) 올라 1만859.72로 거래를 마쳤다. 현재 나스닥100 지수선물은 0.26% 오름세다.코스닥은 전날 대비 14.02p(2.08%) 상승한 688.50를 가리키고 있다.외국인은 1128억원, 기관은 1987억원 각각 순매수했다. 개인은 3164억원 순매도했다.코스닥 시가총액 상위 10개 종목이 모두 올랐다. HLB(6.24%), 엘앤에프(3.88%), 리노공업(3.63%), 셀트리온헬스케어(2.82%), 셀트리온제약(2.37%), 펄어비스(1.86%), 천보(1.24%), 에코프로(0.88%), 에코프로비엠(0.56%), 카카오게임즈(0.53%) 등은 상승했다.상승 업종은 비금속(5.34%), 출판·매체복제(4.15%), 반도체(3.42%), 운송장비·부품(3.28%), 금속(2.84%) 등이다. 하락 업종은 통신서비스(-0.01%)다.서울외환시장에서 달러·원 환율은 전 거래일보다 0.1원 내린 1439.7원에 마감했다.장초반 달러 강세 압력 완화에 1429원까지 하락했지만, 엔화가 재차 변동성 확대되어 강세전환하고 위안화도 약세 기록한 영향에 하락폭을 반납했다."
sentiment_predict(today_news)

1/1 [==============================] - 1s 532ms/step
50.58% 확률로 부정 리뷰입니다.

